In [1]:
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings(action="ignore",category=DeprecationWarning)
warnings.filterwarnings(action="ignore",category=FutureWarning)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
import gc
import copy
from sklearn.model_selection import train_test_split, GroupKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
from datetime import datetime

from sklearn.metrics import roc_auc_score


from keras.layers import Dense, Input, Activation
from keras.layers import BatchNormalization,Add,Dropout
from keras.optimizers import Adam
from keras.models import Model, load_model
from keras import callbacks
from keras import backend as K

from time import time
import datetime

Using TensorFlow backend.


In [2]:
folder_path = '../data/IEEE-CIS-Fraud-Detection/'

In [3]:
df_train = pd.read_pickle(f'{folder_path}/df_train2.gzde', compression='gzip')#.iloc[:10000,:]
df_test = pd.read_pickle(f'{folder_path}/df_test2.gzde', compression='gzip')#.iloc[:10000,:]

In [4]:
columns = df_test.columns.tolist()
columns.remove('TransactionID')
columns.remove('TransactionDT')

In [5]:
def create_nn_model(input_shape, num_classes):
    dropout = .2
    inp = Input(shape=(input_shape,))
    x = Dense(2048, activation="relu")(inp)
    x = BatchNormalization()(x)
#     x = Dropout(dropout)(x)
    for i in range(1):
        x = Dense(1024, activation="relu")(x)
        x = BatchNormalization()(x)
#         x = Dropout(dropout)(x)
    x = Dense(512, activation="relu")(x)
    x = BatchNormalization()(x)
#     x = Dropout(dropout)(x)
    x = Dense(128, activation="relu")(x)
    x = BatchNormalization()(x)
#     x = Dropout(dropout)(x)
    
#     out = Dense(1, activation="linear")(x)
    out = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=inp, outputs=[out])
    return model

def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [6]:
# Set up GPU preferences
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 2} ) 
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.6
sess = tf.Session(config=config) 
K.set_session(sess)

In [8]:
fit_params={
    'batch_size':64,
    'epochs':200,
}

In [9]:
def process(folds, df_train,df_test, columns, is_output_feature_importance=1, verbose=0):

#     aucs = list()
    his = []
    training_start_time = time()
    df_feature_importances_i_list = []
    df_valid_pred = pd.DataFrame()
    df_test_pred = pd.DataFrame()
    if type(df_test) != type(None):
        df_test_pred['TransactionID'] = df_test['TransactionID']
    
    X,y = df_train.sort_values('TransactionDT')[columns], df_train.sort_values('TransactionDT')['isFraud']
    if type(df_test) != type(None):
        X_test = df_test.sort_values('TransactionDT')[columns]
        
    for fold, (trn_idx, test_idx) in enumerate(folds.split(X, y)):
        start_time = time()
        if verbose > 1:
            print('Training on fold {}'.format(fold + 1))
        
        clf = nn_model=create_nn_model(X.shape[1], 2)
        clf.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=[auc])
        
        train_history = clf.fit(X.iloc[trn_idx].values, y.iloc[trn_idx].values, validation_data=(X.iloc[test_idx].values, y.iloc[test_idx].values), **fit_params)
        
        y_trn_pred = clf.predict(X.iloc[trn_idx].values) [:,0]
        y_val_pred = clf.predict(X.iloc[test_idx].values)[:,0]
        
        original_index = df_train['TransactionID'].values[test_idx]
        df_valid_pred_i = pd.DataFrame({'TransactionID': original_index, 'predict': y_val_pred, 'fold': np.zeros(y_val_pred.shape[0]) + fold})
        df_valid_pred = pd.concat([df_valid_pred, df_valid_pred_i], axis=0)
        
        y_test_pred = None
        if type(df_test)!=type(None):
            y_test_pred = clf.predict(X_test.values)[:,0]
            df_test_pred_i = pd.DataFrame({fold: y_test_pred})
            df_test_pred = pd.concat([df_test_pred, df_test_pred_i], axis=1)
        
        trn_auc = roc_auc_score(y.iloc[trn_idx].values, y_trn_pred)
        val_auc = roc_auc_score(y.iloc[test_idx].values, y_val_pred)
        
        if is_output_feature_importance:
            perm = PermutationImportance(clf, random_state=42).fit(X.iloc[test_idx].values, y.iloc[test_idx].values)
            df_feature_importances_i2 = eli5.explain_weights_dfs(perm, feature_names=columns, top=len(columns))['feature_importances']
            df_feature_importances_i2 = df_feature_importances_i2.sort_values(by=['feature'])
            df_feature_importances_i2 = df_feature_importances_i2.reset_index(drop=True)
            df_feature_importances_i_list.append(df_feature_importances_i2)
        
#         aucs.append(clf.best_score['valid_1']['auc'])
        his.append({'val_auc':val_auc, 'trn_auc':trn_auc, 'y_val_pred':y_val_pred, 'y_test_pred':y_test_pred, 'test_idx':test_idx})
        if verbose > 0:
            print('Fold {} finished in {}'.format(fold + 1, str(datetime.timedelta(seconds=time() - start_time))))
    his = pd.DataFrame(his)
    
    df_feature_importances = None
    if is_output_feature_importance:
        df_feature_importances = df_feature_importances_i_list[0]
        for idx, df_feature_importances_i in enumerate(df_feature_importances_i_list[1:]):
            df_feature_importances = pd.merge(df_feature_importances, df_feature_importances_i, on='feature', suffixes=('', idx + 1))
            
    df_valid_pred = df_valid_pred.sort_values(by=['TransactionID'])
    df_valid_pred = df_valid_pred.reset_index(drop=True)

    if type(df_test) != type(None):
        df_test_pred = df_test_pred.sort_values(by=['TransactionID'])
        df_test_pred = df_test_pred.reset_index(drop=True)
    
    if verbose > 0:
        print('-' * 30)
        print('Training has finished.')
        print('Total training time is {}'.format(str(datetime.timedelta(seconds=time() - training_start_time))))
        print('Mean AUC:', his.val_auc.mean(), his.trn_auc.mean())
        print('-' * 30)
    return his, df_feature_importances, df_valid_pred, df_test_pred, his.val_auc.mean()

In [10]:
# folds = TimeSeriesSplit(n_splits=5)
# folds = GroupKFold(n_splits=5)
folds = KFold(n_splits=8, shuffle=False, random_state=42)

In [ ]:
his, df_feature_importances, df_valid_pred, df_test_pred, val_metric = process(folds, df_train,df_test, columns, is_output_feature_importance=0, verbose=1)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use tf.cast instead.
Train on 516722 samples, validate on 73818 samples
Epoch 1/200
516722/516722 [==============================] - 75s 145us/step - loss: 0.1235 - auc: 0.7883 - val_loss: 0.1212 - val_auc: 0.8150
Epoch 2/200
516722/516722 [==============================] - 74s 143us/step - loss: 0.1126 - auc: 0.8210 - val_loss: 0.1293 - val_auc: 0.8280
Epoch 3/200
516722/516722 [==============================] - 78s 151us/step - loss: 0.1104 - auc: 0.8320 - val_loss: 0.1067 - val_auc: 0.8346
Epoch 4/200
516722/516722 [==============================] - 72s 138us/step - loss: 0.1088 - auc: 0.8366 - val_loss: 0.1320 - val_auc: 0.8384
Epoch 5/200
516722/516722 [==============================] - 76s 147us/step - loss: 0.1070 - auc: 0.8402 - val_loss: 0.1252 - val_auc: 